# 7장 정규표현식

## 07-1 정규표현식 살펴보기
정규 표현식(Regular Expressions)은 복잡한 문자열을 처리할 때 사용하는 기법으로, 파이썬만의 고유 문법이 아니라 문자열을 처리하는 모든 곳에서 사용한다.
> ※ 정규 표현식은 줄여서 간단히 "정규식"이라고도 말한다.

    주민등록번호를 포함하고 있는 텍스트가 있다. 이 텍스트에 포함된 모든 주민등록번호의 뒷자리를 * 문자로 변경해 보자.

In [1]:
data = """
park 800905-1049118
kim  700905-1059119
"""

result = []
for line in data.split("\n"):
    word_result = []
    for word in line.split(" "):
        if len(word) == 14 and word[:6].isdigit() and word[7:].isdigit():
            word = word[:6] + "-" + "*******"
        word_result.append(word)
    result.append(" ".join(word_result))
print("\n".join(result))


park 800905-*******
kim  700905-*******



### 정규식 사용

In [2]:
import re 

data = """
park 800905-1049118
kim  700905-1059119
"""

pat = re.compile("(\d{6})[-]\d{7}")
print(pat.sub("\g<1>-*******", data))


park 800905-*******
kim  700905-*******



## 07-2 정규 표현식 시작하기

### 메타 문자(정규 표현식의 기초)
> ※ 메타 문자란 원래 그 문자가 가진 뜻이 아닌 특별한 용도로 사용하는 문자를 말한다.  

`. ^ $ * + ? { } [ ] \ | ( ) `


#### 문자 클래스 []
우리가 가장 먼저 살펴볼 메타 문자는 바로 문자 클래스(character class)인 [ ]이다. 문자 클래스로 만들어진 정규식은 `"[ ] 사이의 문자들과 매치"`라는 의미를 갖는다.
> ※ 문자 클래스를 만드는 메타 문자인 [ ] 사이에는 모든 문자가 들어갈 수 있다.

즉, 정규 표현식이 [abc]라면 이 표현식의 의미는 "a, b, c 중 한 개의 문자와 매치"를 뜻한다.  
이해를 돕기 위해 문자열 "a", "before", "dude"가 정규식 [abc]와 어떻게 매치되는지 살펴보자.

- "a"는 정규식과 일치하는 문자인 "a"가 있으므로 매치
- "before"는 정규식과 일치하는 문자인 "b"가 있으므로 매치
- "dude"는 정규식과 일치하는 문자인 a, b, c 중 어느 하나도 포함하고 있지 않으므로 매치되지 않음

[ ] 안의 두 문자 사이에 하이픈(-)을 사용하면 두 문자 사이의 범위(From - To)를 의미한다. 예를 들어 [a-c]라는 정규 표현식은 [abc]와 동일하고 [0-5]는 [012345]와 동일하다.

다음은 하이픈(-)을 사용한 문자 클래스의 사용 예이다.

- [a-zA-Z] : 알파벳 모두
- [0-9] : 숫자

문자 클래스([ ]) 안에는 어떤 문자나 메타 문자도 사용할수 있지만 주의해야 할 메타 문자가 1가지 있다.  
그것은 바로 `^`인데, 문자 클래스 안에 `^` 메타 문자를 사용할 경우에는 반대(not)라는 의미를 갖는다.  
예를 들어 `[^0-9]`라는 정규 표현식은 숫자가 아닌 문자만 매치된다.

**[자주 사용하는 문자 클래스]**  
[0-9] 또는 [a-zA-Z] 등은 무척 자주 사용하는 정규 표현식이다. 이렇게 자주 사용하는 정규식은 별도의 표기법으로 표현할 수 있다. 다음을 기억해 두자.

- `\d` - 숫자와 매치, `[0-9]`와 동일한 표현식이다.
- `\D` - 숫자가 아닌 것과 매치, `[^0-9]`와 동일한 표현식이다.
- `\s` - whitespace 문자와 매치, `[ \t\n\r\f\v]`와 동일한 표현식이다. 맨 앞의 빈 칸은 공백문자(space)를 의미한다.
- `\S` - whitespace 문자가 아닌 것과 매치, `[^ \t\n\r\f\v]`와 동일한 표현식이다.
- `\w` - 문자+숫자(alphanumeric)와 매치, `[a-zA-Z0-9_]`와 동일한 표현식이다.
- `\W` - 문자+숫자(alphanumeric)가 아닌 문자와 매치, `[^a-zA-Z0-9_]`와 동일한 표현식이다.  
  
대문자로 사용된 것은 소문자의 반대임을 추측할 수 있다.

#### Dot(.)
정규 표현식의 Dot(.) 메타 문자는 줄바꿈 문자인 `\n`을 제외한 모든 문자와 매치됨을 의미한다.

> ※ 나중에 배우겠지만 정규식을 작성할 때 re.DOTALL 옵션을 주면 `\n` 문자와도 매치된다.

다음 정규식을 보자.

`a.b`

위 정규식의 의미는 다음과 같다.

> "a + 모든문자 + b"

즉 a와 b라는 문자 사이에 어떤 문자가 들어가도 모두 매치된다는 의미이다.

이해를 돕기 위해 문자열 "aab", "a0b", "abc"가 정규식 `a.b`와 어떻게 매치되는지 살펴보자.

- "aab"는 가운데 문자 "a"가 모든 문자를 의미하는 `.`과 일치하므로 정규식과 매치된다.
- "a0b"는 가운데 문자 "0"가 모든 문자를 의미하는 `.`과 일치하므로 정규식과 매치된다.
- "abc"는 "a"문자와 "b"문자 사이에 어떤 문자라도 하나는있어야 하는 이 정규식과 일치하지 않으므로 매치되지 않는다.

#### 반복(*)
다음 정규식을 보자.

`ca*t`
이 정규식에는 반복을 의미하는 `*` 메타 문자가 사용되었다. 여기에서 사용한 `*`은 `*` 바로 앞에 있는 문자 a가 0부터 무한대로 반복될 수 있다는 의미이다.

> ※ 여기에서 `*` 메타 문자의 반복 개수가 무한대라고 표현했는데 사실 메모리 제한으로 2억 개 정도만 가능하다고 한다.

즉 다음과 같은 문자열이 모두 매치된다.

정규식|문자열|Match 여부|설명
-|-|-|-
ca*t|ct|Yes|"a"가 0번 반복되어 매치
ca*t|cat|Yes|"a"가 0번 이상 반복되어 매치 (1번 반복)
ca*t|caaat|Yes|"a"가 0번 이상 반복되어 매치 (3번 반복)

#### 반복(+)
반복을 나타내는 또 다른 메타 문자로 +가 있다. `+`는 최소 1번 이상 반복될 때 사용한다. 즉 `*`가 반복 횟수 0부터라면 `+`는 반복 횟수 1부터인 것이다.

다음 정규식을 보자.

`ca+t`

위 정규식의 의미는 다음과 같다.

> "c + a(1번 이상 반복) + t"

위 정규식에 대한 매치여부는 다음 표와 같다.

정규식|문자열|Match 여부|설명
-|-|-|-
ca+t|ct|No|"a"가 0번 반복되어 매치되지 않음
ca+t|cat|Yes|"a"가 1번 이상 반복되어 매치 (1번 반복)
ca+t|caaat|Yes|"a"가 1번 이상 반복되어 매치 (3번 반복)

#### 반복({m, n}, ?)
여기에서 잠깐 생각해 볼 게 있다. 반복 횟수를 3회만 또는 1회부터 3회까지만으로 제한하고 싶을 수도 있지 않을까?

{ } 메타 문자를 사용하면 반복 횟수를 고정할 수 있다. {m, n} 정규식을 사용하면 반복 횟수가 m부터 n까지 매치할 수 있다. 또한 m 또는 n을 생략할 수도 있다. 만약 {3,}처럼 사용하면 반복 횟수가 3 이상인 경우이고 {,3}처럼 사용하면 반복 횟수가 3 이하를 의미한다. 생략된 m은 0과 동일하며, 생략된 n은 무한대(2억 개 미만)의 의미를 갖는다.

> ※ {1,}은 `+`와 동일하고, {0,}은 `*`와 동일하다.

1. {m}
`ca{2}t`

위 정규식의 의미는 다음과 같다.

> "c + a(반드시 2번 반복) + t"

위 정규식에 대한 매치여부는 다음 표와 같다.

정규식|문자열|Match 여부|설명
-|-|-|-
ca{2}t|cat|No|"a"가 1번만 반복되어 매치되지 않음
ca{2}t|caat|Yes|"a"가 2번 반복되어 매치

2. {m, n}

`ca{2,5}t`

위 정규식의 의미는 다음과 같다:

> "c + a(2~5회 반복) + t"

위 정규식에 대한 매치여부는 다음 표와 같다.

정규식|문자열|Match 여부|설명
-|-|-|-
ca{2,5}t|cat|No|"a"가 1번만 반복되어 매치되지 않음
ca{2,5}t|caat|Yes|"a"가 2번 반복되어 매치
ca{2,5}t|caaaaat|Yes|"a"가 5번 반복되어 매치

3. `?`

반복은 아니지만 이와 비슷한 개념으로 `?` 이 있다. `?` 메타문자가 의미하는 것은 `{0, 1}` 이다.

다음 정규식을 보자.

`ab?c`

위 정규식의 의미는 다음과 같다:

> "a + b(있어도 되고 없어도 된다) + c"

위 정규식에 대한 매치여부는 다음 표와 같다.

정규식|문자열|Match 여부|설명
-|-|-|-
ab?c|abc|Yes|"b"가 1번 사용되어 매치
ab?c|ac|Yes|"b"가 0번 사용되어 매치

즉 b 문자가 있거나 없거나 둘 다 매치되는 경우이다.

### 파이썬에서 정규 표현식을 지원하는 re 모듈

In [3]:
import re
p = re.compile('ab*')

re.compile을 사용하여 정규 표현식(위 예에서는 `ab*`)을 컴파일한다. re.compile의 결과로 돌려주는 객체 p(컴파일된 패턴 객체)를 사용하여 그 이후의 작업을 수행할 것이다.
- ※ 정규식을 컴파일할 때 특정 옵션을 주는 것도 가능한데, 이에 대해서는 뒤에서 자세히 살펴본다.
- ※ 패턴이란 정규식을 컴파일한 결과이다.

### 정규식을 이용한 문자열 검색
컴파일된 패턴 객체는 다음과 같은 4가지 메서드를 제공한다.

Method|목적
-|-
match()|문자열의 처음부터 정규식과 매치되는지 조사한다.
search()|문자열 전체를 검색하여 정규식과 매치되는지 조사한다.
findall()|정규식과 매치되는 모든 문자열(substring)을 리스트로 돌려준다.
finditer()|정규식과 매치되는 모든 문자열(substring)을 반복 가능한 객체로 돌려준다.

match, search는 정규식과 매치될 때는 match 객체를 돌려주고, 매치되지 않을 때는 None을 돌려준다.
> ※ match 객체란 정규식의 검색 결과로 돌려주는 객체이다.

In [4]:
p = re.compile('[a-z]+')

#### match

In [5]:
m = p.match('python')
print(m)

<re.Match object; span=(0, 6), match='python'>


In [6]:
m = p.match('3 python')
print(m)

None


match의 결과로 match 객체 또는 None을 돌려주기 때문에 파이썬 정규식 프로그램은 보통 다음과 같은 흐름으로 작성한다.

```python
p = re.compile(정규표현식)
m = p.match('string goes here')
if m:
    print('Match found: ', m.group())
else:
    print('No match')
```

즉 match의 결괏값이 있을 때만 그다음 작업을 수행하겠다는 것이다.

#### search

In [7]:
m = p.search('python')
print(m)

<re.Match object; span=(0, 6), match='python'>


In [8]:
m = p.search('3 python')
print(m)

<re.Match object; span=(2, 8), match='python'>


"3 python" 문자열의 첫 번째 문자는 "3"이지만 search는 문자열의 처음부터 검색하는 것이 아니라 문자열 전체를 검색하기 때문에 "3 " 이후의 "python" 문자열과 매치된다.

#### findall

In [9]:
result = p.findall('life is too short')
print(result)

['life', 'is', 'too', 'short']


"life is too short" 문자열의 'life', 'is', 'too', 'short' 단어를 각각 `[a-z]+` 정규식과 매치해서 리스트로 돌려준다.

#### finditer

In [11]:
result = p.finditer('life is too short')
print(result)
for r in result: print(r)

<re.Match object; span=(0, 4), match='life'>
<re.Match object; span=(5, 7), match='is'>
<re.Match object; span=(8, 11), match='too'>
<re.Match object; span=(12, 17), match='short'>


finditer는 findall과 동일하지만 그 결과로 반복 가능한 객체(iterator object)를 돌려준다. 반복 가능한 객체가 포함하는 각각의 요소는 match 객체이다.

### match 객체의 메서드
- 어떤 문자열이 매치되었는가?
- 매치된 문자열의 인덱스는 어디서부터 어디까지인가?

method|목적
-|-
group()|매치된 문자열을 돌려준다.
start()|매치된 문자열의 시작 위치를 돌려준다.
end()|매치된 문자열의 끝 위치를 돌려준다.
span()|매치된 문자열의 (시작, 끝)에 해당하는 튜플을 돌려준다.

In [12]:
m = p.match('python')
m.group()

'python'

In [13]:
m.start()

0

In [14]:
m.end()

6

In [15]:
m.span()

(0, 6)

In [16]:
m = p.search('3 python')
m.group()

'python'

In [17]:
m.start()

2

In [18]:
m.end()

8

In [19]:
m.span()

(2, 8)

### [모듈 단위로 수행하기]
re 모듈은 이것을 좀 축약한 형태로 사용할 수 있는 방법을 제공한다. 

In [21]:
p = re.compile('[a-z]+')
m = p.match('python')
m

<re.Match object; span=(0, 6), match='python'>

In [23]:
m = re.match('[a-z]+', 'python')
m

<re.Match object; span=(0, 6), match='python'>

### 컴파일 옵션
- DOTALL(S) - `.` 이 줄바꿈 문자를 포함하여 모든 문자와 매치할 수 있도록 한다.
- IGNORECASE(I) - 대소문자에 관계없이 매치할 수 있도록 한다.
- MULTILINE(M) - 여러줄과 매치할 수 있도록 한다. (`^`, `$` 메타문자의 사용과 관계가 있는 옵션이다)
- VERBOSE(X) - verbose 모드를 사용할 수 있도록 한다. (정규식을 보기 편하게 만들수 있고 주석등을 사용할 수 있게된다.)

옵션을 사용할 때는 `re.DOTALL`처럼 전체 옵션 이름을 써도 되고 `re.S`처럼 약어를 써도 된다.

#### DOTALL, S

In [24]:
import re
p = re.compile('a.b')
m = p.match('a\nb')
print(m)

None


In [25]:
p = re.compile('a.b', re.DOTALL)
m = p.match('a\nb')
print(m)

<re.Match object; span=(0, 3), match='a\nb'>


In [26]:
p = re.compile('a.b', re.S)
m = p.match('a\nb')
print(m)

<re.Match object; span=(0, 3), match='a\nb'>


#### IGNORECASE, I

In [27]:
p = re.compile('[a-z]+', re.I)
p.match('python')

<re.Match object; span=(0, 6), match='python'>

In [28]:
p.match('Python')

<re.Match object; span=(0, 6), match='Python'>

In [29]:
p.match('PYTHON')

<re.Match object; span=(0, 6), match='PYTHON'>

#### MULTILINE, M
`^`는 문자열의 처음을 의미하고, `$`는 문자열의 마지막을 의미

In [30]:
p = re.compile('^python\s\w+')

data = '''python one
life is too short
python two
you need python
python three'''

print(p.findall(data))

['python one']


정규식 `^python\s\w+`은 python이라는 문자열로 시작하고 그 뒤에 whitespace, 그 뒤에 단어가 와야 한다는 의미

`^` 메타 문자에 의해 python이라는 문자열을 사용한 첫 번째 줄만 매치

`^` 메타 문자를 문자열 전체의 처음이 아니라 각 라인의 처음으로 인식시키고 싶은 경우

In [31]:
p = re.compile('^python\s\w+', re.M)

data = '''python one
life is too short
python two
you need python
python three'''

print(p.findall(data))

['python one', 'python two', 'python three']


`re.MULTILINE` 옵션은 `^`, `$` 메타 문자를 문자열의 각 줄마다 적용해 주는 것이다.

#### VERBOSE, X
정규식을 주석 또는 줄 단위로 구분할 수 있다

In [32]:
charref = re.compile(r'&[#](0[0-7]+|[0-9]+|x[0-9a-fA-F]+);')

In [33]:
charref = re.compile(r"""
 &[#]                # Start of a numeric entity reference
 (
     0[0-7]+         # Octal form
   | [0-9]+          # Decimal form
   | x[0-9a-fA-F]+   # Hexadecimal form
 )
 ;                   # Trailing semicolon
""", re.VERBOSE)

첫 번째와 두 번째 예를 비교해 보면 컴파일된 패턴 객체인 charref는 모두 동일한 역할을 한다. 하지만 정규식이 복잡할 경우 두 번째처럼 주석을 적고 여러 줄로 표현하는 것이 훨씬 가독성이 좋다는 것을 알 수 있다.

`re.VERBOSE` 옵션을 사용하면 문자열에 사용된 whitespace는 컴파일할 때 제거된다(단 [ ] 안에 사용한 whitespace는 제외). 그리고 줄 단위로 #기호를 사용하여 주석문을 작성할 수 있다.

### 백슬래시 문제
예를 들어 어떤 파일 안에 있는 `"\section"` 문자열을 찾기 위한 정규식을 만든다고 가정해 보자.

`\section`

이 정규식은 `\s` 문자가 whitespace로 해석되어 의도한 대로 매치가 이루어지지 않는다.

위 표현은 다음과 동일한 의미이다.

`[ \t\n\r\f\v]ection`

의도한 대로 매치하고 싶다면 다음과 같이 변경해야 한다.

`\\section`

즉 위 정규식에서 사용한 `\` 문자가 문자열 자체임을 알려 주기 위해 백슬래시 2개를 사용하여 이스케이프 처리를 해야 한다.

따라서 위 정규식을 컴파일하려면 다음과 같이 작성해야 한다.

```python
>>> p = re.compile('\\section')
```

그런데 여기에서 또 하나의 문제가 발견된다. 위처럼 정규식을 만들어서 컴파일하면 실제 파이썬 정규식 엔진에는 파이썬 문자열 리터럴 규칙에 따라 `\\`이 `\`로 변경되어 `\section`이 전달된다.

> ※ 이 문제는 위와 같은 정규식을 파이썬에서 사용할 때만 발생한다(파이썬의 리터럴 규칙). 유닉스의 grep, vi 등에서는 이러한 문제가 없다.

결국 정규식 엔진에 `\\` 문자를 전달하려면 파이썬은 `\\\\`처럼 백슬래시를 4개나 사용해야 한다.

> ※ 정규식 엔진은 정규식을 해석하고 수행하는 모듈이다.

```python
>>> p = re.compile('\\\\section')
```

이렇게 해야만 원하는 결과를 얻을 수 있다. 하지만 너무 복잡하지 않은가?

만약 위와 같이 `\`를 사용한 표현이 계속 반복되는 정규식이라면 너무 복잡해서 이해하기 쉽지않을 것이다. 이러한 문제로 인해 파이썬 정규식에는 Raw String 규칙이 생겨나게 되었다. 즉 컴파일해야 하는 정규식이 Raw String임을 알려 줄 수 있도록 파이썬 문법을 만든 것이다. 그 방법은 다음과 같다.

```python
>>> p = re.compile(r'\\section')
```

위와 같이 정규식 문자열 앞에 r 문자를 삽입하면 이 정규식은 Raw String 규칙에 의하여 백슬래시 2개 대신 1개만 써도 2개를 쓴 것과 동일한 의미를 갖게 된다.

> ※ 만약 백슬래시를 사용하지 않는 정규식이라면 r의 유무에 상관없이 동일한 정규식이 될 것이다.

## 07-3 강력한 정규 표현식의 세계로

### 메타문자

#### |
`|` 메타 문자는 or과 동일한 의미로 사용된다. `A|B`라는 정규식이 있다면 A 또는 B라는 의미가 된다.

In [34]:
p = re.compile('Crow|Servo')
m = p.match('CrowHello')
print(m)

<re.Match object; span=(0, 4), match='Crow'>


#### ^
`^` 메타 문자는 문자열의 맨 처음과 일치함을 의미한다. 앞에서 살펴본 컴파일 옵션 `re.MULTILINE`을 사용할 경우에는 여러 줄의 문자열일 때 각 줄의 처음과 일치하게 된다.

In [35]:
print(re.search('^Life', 'Life is too short'))
print(re.search('^Life', "My Life"))

<re.Match object; span=(0, 4), match='Life'>
None


#### $
`$` 메타 문자는 `^` 메타 문자와 반대의 경우이다. 즉 `$`는 문자열의 끝과 매치함을 의미한다.

In [36]:
print(re.search('short$', 'Life is too short'))
print(re.search('short$', 'Life is too shor, you need python'))

<re.Match object; span=(12, 17), match='short'>
None


#### \A
`\A`는 문자열의 처음과 매치됨을 의미한다. `^` 메타 문자와 동일한 의미이지만 `re.MULTILINE` 옵션을 사용할 경우에는 다르게 해석된다. `re.MULTILINE` 옵션을 사용할 경우 `^`은 각 줄의 문자열의 처음과 매치되지만 `\A`는 줄과 상관없이 전체 문자열의 처음하고만 매치된다.

#### \Z
`\Z`는 문자열의 끝과 매치됨을 의미한다. 이것 역시 `\A`와 동일하게 `re.MULTILINE` 옵션을 사용할 경우 `$` 메타 문자와는 달리 전체 문자열의 끝과 매치된다.

#### \b
`\b`는 단어 구분자(Word boundary)이다. 보통 단어는 whitespace에 의해 구분된다.

In [37]:
p = re.compile(r'\bclass\b')
print(p.search('no class at all'))

<re.Match object; span=(3, 8), match='class'>


`\bclass\b` 정규식은 앞뒤가 whitespace로 구분된 class라는 단어와 매치됨을 의미한다. 따라서 no class at all의 class라는 단어와 매치됨을 확인할 수 있다.

In [39]:
print(p.search('the declassified algorithm'))

None


위 예의 the declassified algorithm 문자열 안에도 class 문자열이 포함되어 있긴 하지만 whitespace로 구분된 단어가 아니므로 매치되지 않는다.

In [38]:
print(p.search('one subclass is'))

None


subclass 문자열 역시 class 앞에 sub 문자열이 더해져 있으므로 매치되지 않음을 알 수 있다.

`\b` 메타 문자를 사용할 때 주의해야 할 점이 있다. `\b`는 파이썬 리터럴 규칙에 의하면 백스페이스(BackSpace)를 의미하므로 백스페이스가 아닌 단어 구분자임을 알려 주기 위해 `r'\bclass\b'`처럼 Raw string임을 알려주는 기호 r을 반드시 붙여 주어야 한다.

#### \B
`\B` 메타 문자는 `\b` 메타 문자와 반대의 경우이다. 즉 whitespace로 구분된 단어가 아닌 경우에만 매치된다.

In [40]:
p = re.compile(r'\Bclass\B')
print(p.search('no class at all'))
print(p.search('the declassified algorithm'))
print(p.search('one subclass is'))

None
<re.Match object; span=(6, 11), match='class'>
None


class 단어의 앞뒤에 whitespace가 하나라도 있는 경우에는 매치가 안 되는 것을 확인할 수 있다.

### 그루핑

ABC 문자열이 계속해서 반복되는지 조사하는 정규식을 작성

`(ABC)+`

그룹을 만들어 주는 메타 문자는 바로 `( )`이다.

In [41]:
p = re.compile('(ABC)+')
m = p.search('ABCABCABC OK?')
print(m)
print(m.group())

<re.Match object; span=(0, 9), match='ABCABCABC'>
ABCABCABC


In [44]:
p = re.compile(r"\w+\s+\d+[-]\d+[-]\d+")
m = p.search("park 010-1234-1234")

`\w+\s+\d+[-]\d+[-]\d+`은 `이름 + " " + 전화번호` 형태의 문자열을 찾는 정규식이다. 그런데 이렇게 매치된 문자열 중에서 이름만 뽑아내고 싶다면 어떻게 해야 할까?

보통 반복되는 문자열을 찾을 때 그룹을 사용하는데, 그룹을 사용하는 보다 큰 이유는 위에서 볼 수 있듯이 매치된 문자열 중에서 특정 부분의 문자열만 뽑아내기 위해서인 경우가 더 많다.

In [47]:
p = re.compile(r"(\w+)\s+\d+[-]\d+[-]\d+")
m = p.search("park 010-1234-1234")
print(m.group(1))

park


이름에 해당하는 `\w+` 부분을 그룹 `(\w+)`으로 만들면 match 객체의 group(인덱스) 메서드를 사용하여 그루핑된 부분의 문자열만 뽑아낼 수 있다. group 메서드의 인덱스는 다음과 같은 의미를 갖는다.

group(인덱스)|설명
-|-
group(0)|매치된 전체 문자열
group(1)|첫 번째 그룹에 해당되는 문자열
group(2)|두 번째 그룹에 해당되는 문자열
group(n)|n 번째 그룹에 해당되는 문자열

In [48]:
p = re.compile(r"(\w+)\s+(\d+[-]\d+[-]\d+)")
m = p.search("park 010-1234-1234")
print(m.group(2))

010-1234-1234


이번에는 전화번호 부분을 추가로 그룹 `(\d+[-]\d+[-]\d+)`로 만들었다. 이렇게 하면 group(2)처럼 사용하여 전화번호만 뽑아낼 수 있다.

In [49]:
p = re.compile(r"(\w+)\s+((\d+)[-]\d+[-]\d+)")
m = p.search("park 010-1234-1234")
print(m.group(3))

010


위 예에서 볼 수 있듯이 `(\w+)\s+((\d+)[-]\d+[-]\d+)`처럼 그룹을 중첩되게 사용하는 것도 가능하다. 그룹이 중첩되어 있는 경우는 바깥쪽부터 시작하여 안쪽으로 들어갈수록 인덱스가 증가한다.

#### 그루핑된 문자열 재참조하기
그룹의 또 하나 좋은 점은 한 번 그루핑한 문자열을 재참조(Backreferences)할 수 있다는 점이다. 다음 예를 보자.

In [50]:
p = re.compile(r'(\b\w+)\s+\1')
p.search('Paris in the the spring').group()

'the the'

정규식 `(\b\w+)\s+\1`은 `(그룹) + " " + 그룹과 동일한 단어`와 매치됨을 의미한다. 이렇게 정규식을 만들게 되면 2개의 동일한 단어를 연속적으로 사용해야만 매치된다. 이것을 가능하게 해주는 것이 바로 재참조 메타 문자인 `\1`이다. `\1`은 정규식의 그룹 중 첫 번째 그룹을 가리킨다.

※ 두 번째 그룹을 참조하려면 `\2`를 사용하면 된다.

#### 그루핑된 문자열에 이름 붙이기
정규식 안에 그룹이 무척 많아진다고 가정해 보자. 예를 들어 정규식 안에 그룹이 10개 이상만 되어도 매우 혼란스러울 것이다. 거기에 더해 정규식이 수정되면서 그룹이 추가, 삭제되면 그 그룹을 인덱스로 참조한 프로그램도 모두 변경해 주어야 하는 위험도 갖게 된다.

만약 그룹을 인덱스가 아닌 이름(Named Groups)으로 참조할 수 있다면 어떨까? 그렇다면 이런 문제에서 해방되지 않을까?

이러한 이유로 정규식은 그룹을 만들 때 그룹 이름을 지정할 수 있게 했다. 그 방법은 다음과 같다.

`(?P<name>\w+)\s+((\d+)[-]\d+[-]\d+)`

위 정규식은 앞에서 본 이름과 전화번호를 추출하는 정규식이다. 기존과 달라진 부분은 다음과 같다.

> `(\w+)` --> `(?P<name>\w+)`

대단히 복잡해진 것처럼 보이지만 `(\w+)`라는 그룹에 name이라는 이름을 붙인 것에 불과하다. 여기에서 사용한 `(?...)` 표현식은 정규 표현식의 확장 구문이다. 이 확장 구문을 사용하기 시작하면 가독성이 상당히 떨어지긴 하지만 반면에 강력함을 갖게 된다.

그룹에 이름을 지어 주려면 다음과 같은 확장 구문을 사용해야 한다.

In [52]:
p = re.compile(r"(?P<name>\w+)\s+((\d+)[-]\d+[-]\d+)")
m = p.search("park 010-1234-1234")
print(m.group("name"))

park


위 예에서 볼 수 있듯이 name이라는 그룹 이름으로 참조할 수 있다.

그룹 이름을 사용하면 정규식 안에서 재참조하는 것도 가능하다.

In [53]:
p = re.compile(r'(?P<word>\b\w+)\s+(?P=word)')
p.search('Paris in the the spring').group()

'the the'

위 예에서 볼 수 있듯이 재참조할 때에는 (?P=그룹이름)이라는 확장 구문을 사용해야 한다.